In [4]:
with open("test.html", "r") as f:
    all_html_list = f.readlines()
    PAGE_DICT = {
        idx: content for idx, content in enumerate(all_html_list)
    }

def get_page(test_page_id):
    return PAGE_DICT[test_page_id] if test_page_id in PAGE_DICT else 0

def extract_page_features(page_features, features, test_page):
    page_content = get_page(test_page)
    print(f"page_content: {page_content}")
    page_elements = []
    for i in page_content.split("==page"):
        if "==" in i:
            page_elements.append(int(i.split("==")[0]))
    for i in page_elements:
        page_features[i] = 1

In [6]:

page_features = []
extract_page_features(page_features, [], 3)
print(page_features)

page_content: <!--Script,location--><script nonce=123>location="javascript:'<script src=http://127.0.0.1:8082/blocked_http.js$$$?sign=3 nonce=123><\/script>'"</script>

[]


In [10]:
import json
from features import features

conf = json.load(open("../conf.json", "r"))
generation_list = conf["[JS]"] + conf["[SCHEME]"] + conf["[BLOB_HTML]"] + \
                    conf["[BLOB_JS]"] + conf["[HTML]"]
exist_in_feature_count = 0
non_existed_set = set(generation_list)
for item in generation_list:
    if item in features["page"]:
        exist_in_feature_count += 1
        non_existed_set.remove(item)

print(f"total: {len(generation_list)}")
print(f"exist_in_feature_count: {exist_in_feature_count}")
print(f"exist_in_feature_rate: {exist_in_feature_count / len(generation_list)}")
print(f"non_existed_set: {non_existed_set}")

total: 57
exist_in_feature_count: 56
exist_in_feature_rate: 0.9824561403508771
non_existed_set: {"fetch('[REPORT_API]')"}


In [1]:
elem_dict = {}
with open("features.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        elem_num = len(line.split(","))
        if elem_num not in elem_dict:
            elem_dict[elem_num] = 0
        elem_dict[elem_num] += 1
print(elem_dict)

{109: 2700, 110: 1800}


In [9]:
with open("logs.txt", "r") as f:
    lines = f.readlines()
    for idx in range(len(lines)):
        line = lines[idx]
        if line.startswith("[*] test case #"):
            # executed: [], non-executed: ['Chrome', 'Firefox', 'Safari']
            # executed: ['Chrome', 'Firefox', 'Safari'], non-executed: []
            executed = line.split(", non-executed: ")[0].split("executed:")[1].strip().split(",")
            non_executed = line.split("non-executed: ")[1].strip().split(",")
            if not ((len(executed) == 3 and len(non_executed) == 1) or (len(executed) == 1 and len(non_executed) == 3)):
                print(f"idx: {idx}, executed: {executed}, non-executed: {non_executed}")

idx: 1612, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 1624, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7748, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7760, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7772, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7776, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7788, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7812, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7852, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7880, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7924, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7956, executed: ["['Chrome'", " 'Safari']"], non-executed: ["['Firefox']"]
idx: 7960, executed: ["['Chrome'", " 'Sa

In [10]:
from features import col_names
import pandas as pd

df = pd.read_csv("features.txt", names=col_names)
inconsistent_df = df[df["label"] == 1]
print(inconsistent_df)

        CSP-directive:script=src  CSP-directive:Default-src  \
403                            0                          0   
406                            0                          0   
1937                           0                          0   
1940                           0                          0   
1943                           0                          0   
...                          ...                        ...   
103129                         0                          0   
103213                         0                          0   
103218                         0                          0   
103427                         0                          0   
103434                         0                          0   

        CSP-directive:default-src  CSP-directive:script-src  \
403                             0                         0   
406                             0                         0   
1937                            0                     

In [2]:
count = 0
with open("result.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith("inconsist leaf!"):
            count += 1
print(count)

73


In [2]:
results = []
inconsist_idx = 0
with open("result.txt", "r") as f:
    lines = f.readlines()
    for idx in range(len(lines)):
        line = lines[idx]
        if line.startswith("inconsist leaf!"):
            next_line = lines[idx + 1]
            test_case_id = int(next_line.split(",")[1])
            csp = next_line.split(",")[-1]
            execution_result = lines[idx + 2]
            with open("../test.html", "r") as fin:
                for html_line in fin.readlines():
                    if f"sign={test_case_id}" in html_line:
                        test_page = html_line
                        break
            results.append({
                "idx": inconsist_idx,
                "test_case_id": test_case_id,
                "csp": csp,
                "execution_result": execution_result,
                "test_page": test_page,
            })
            inconsist_idx += 1

import json
with open("result.json", "w") as f:
    json.dump(results, f, indent=2)
